# New Section

In [2]:
!pip install transformers datasets



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 10.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is

In [3]:
from datasets import Dataset

data = {
    'text': [
        "Dear user, your account has been compromised. Click here to reset your password.",  # phishing
        "Important update from your bank. Please verify your identity immediately.",         # phishing
        "Meeting at 3 PM today in conference room B.",                                      # normal
        "Happy Birthday! Wishing you a wonderful day filled with joy.",                     # normal
        "Your package could not be delivered. Provide your payment info to reschedule.",     # phishing
        "Weekly newsletter: Top 10 travel destinations for 2024.",                          # normal
    ],
    'label': [1, 1, 0, 0, 1, 0]  # 1 = phishing, 0 = normal
}

dataset = Dataset.from_dict(data)
dataset = dataset.train_test_split(test_size=0.3)
train_dataset = dataset['train']
test_dataset = dataset['test']



In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

# 사용할 사전학습 모델 이름
model_name = "distilbert-base-uncased"

# 1. 텍스트를 숫자로 바꿔주는 tokenizer 불러오기
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 2. 텍스트 데이터셋을 tokenizer를 이용해서 숫자 변환
def tokenize_function(example):
    return tokenizer(example['text'], truncation=True)

train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# 3. 분류용 모델 (2개의 클래스: phishing / normal) 불러오기
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/4 [00:00<?, ? examples/s]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
from transformers import TrainingArguments, Trainer

# W&B 끄고 훈련 설정
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    report_to="none",        # <= W&B 끄는 옵션 추가!
)

# Trainer 설정
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

# 훈련 시작
trainer.train()


ValueError: expected sequence of length 18 at dim 1 (got 13)

In [9]:
from datasets import Dataset

# 이메일 데이터 준비
data = {
    'text': [
        "Dear user, your account has been compromised. Click here to reset your password.",  # phishing
        "Important update from your bank. Please verify your identity immediately.",         # phishing
        "Meeting at 3 PM today in conference room B.",                                      # normal
        "Happy Birthday! Wishing you a wonderful day filled with joy.",                     # normal
        "Your package could not be delivered. Provide your payment info to reschedule.",     # phishing
        "Weekly newsletter: Top 10 travel destinations for 2024.",                          # normal
    ],
    'label': [1, 1, 0, 0, 1, 0]  # 1 = phishing, 0 = normal
}

# 데이터셋 만들기
dataset = Dataset.from_dict(data)
dataset = dataset.train_test_split(test_size=0.3)
train_dataset = dataset['train']
test_dataset = dataset['test']


In [10]:
# Tokenizer 다시 불러오기
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

# 텍스트를 숫자로 변환
def tokenize_function(example):
    return tokenizer(example['text'], truncation=True)

train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/4 [00:00<?, ? examples/s]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

In [11]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
# Tokenizer 다시 불러오기
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

# 텍스트를 숫자로 변환 (padding=True 추가!!)
def tokenize_function(example):
    return tokenizer(example['text'], padding="max_length", truncation=True)

# 토크나이징 적용
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/4 [00:00<?, ? examples/s]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

In [14]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    report_to="none",  # W&B 끄기
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

trainer.train()


Step,Training Loss


TrainOutput(global_step=3, training_loss=0.6601006189982096, metrics={'train_runtime': 70.0195, 'train_samples_per_second': 0.171, 'train_steps_per_second': 0.043, 'total_flos': 1589608783872.0, 'train_loss': 0.6601006189982096, 'epoch': 3.0})

In [16]:
# 새 이메일 문장들
texts = [
    "Update your payment information now to avoid service interruption.",  # 피싱 의심
    "Let's catch up over coffee next week!",                                # 정상 대화
    "Urgent: Your account has been suspended. Click here to verify.",       # 피싱 의심
    "Congratulations on your work anniversary!"                             # 정상 대화
]

# 입력 데이터 숫자로 변환
inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")

# 모델에게 예측 요청
outputs = model(**inputs)

# 예측 결과 뽑기
predictions = outputs.logits.argmax(dim=1)

# 예측 결과 출력
print(predictions)


tensor([1, 0, 0, 1])
